# 正規線形モデルによるデータ解析

Type2 ANOVAと呼ばれる検定の仕組みを説明


In [3]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

## モデル選択と「フツーの」分散分析(=Type1 ANOVA)の違い

In [40]:
df = pd.read_csv('./data/data0_linearModel.csv')
df['medicine'] = df.medicine.map(dict(yes=1, no=0))
model_a = ols('length ~ food + medicine', data=df).fit()
model_m = ols('length ~ medicine', data=df).fit()
model_f = ols('length ~ food', data=df).fit()
print('##### フツーの分散分析 #####')
print(sm.stats.anova_lm(model_a, typ=1))

##### フツーの分散分析 #####
            df        sum_sq       mean_sq           F        PR(>F)
food       1.0  91095.718662  91095.718662  813.637142  5.774972e-49
medicine   1.0  64103.316067  64103.316067  572.549837  1.772710e-42
Residual  97.0  10860.227805    111.961111         NaN           NaN


In [41]:
# モデル選択
## 薬の有無が体長にもたらす影響検定
print('length ~ medicine')
print('length ~ food + medicine')
print(sm.stats.anova_lm(model_m, model_a, typ=1))
## 餌の量が体長にもたらす影響検定
print('length ~ food')
print('length ~ food + medicine')
print(sm.stats.anova_lm(model_f, model_a, typ=1))

length ~ medicine
length ~ food + medicine
   df_resid            ssr  df_diff        ss_diff          F        Pr(>F)
0      98.0  117921.859389      0.0            NaN        NaN           NaN
1      97.0   10860.227805      1.0  107061.631585  956.23945  4.941231e-52
length ~ food
length ~ food + medicine
   df_resid           ssr  df_diff       ss_diff           F        Pr(>F)
0      98.0  74963.543872      0.0           NaN         NaN           NaN
1      97.0  10860.227805      1.0  64103.316067  572.549837  1.772710e-42


- 最後に入れた説明変数(medicine)の検定結果は、モデル選択でも「フツーの」分散分析(=Type1 ANOVA)でも変わらない
- 最初に入れた説明変数(food)の検定結果は、モデル選択と「フツーの」分散分析(=Type1 ANOVA)で変化する(説明変数が3つい上ある場合、途中に入れた説明変数も同様になる)

## Type1 ANOVAについて

説明変数を入れていないmodelとfood説明変数のmodelを比較することで餌の影響を検定すると、「フツーの」分散分析と結果が一致する.

しかし、ここでは薬の有無という説明変数が入っていないため、薬の影響を完全に無視してしまっている

- Summary
    - 餌の量による影響の大きさは model1とmodel2を比較することにより計算される
        - ここでは、薬の有無は無視して餌の量を評価している
    - 説明変数が3つい上ある場合も同様に説明変数を入れた順番によって影響の大小が変わる

$$ model1: 魚の体長 = 切片 + 誤差1 $$
$$ model2: 魚の体長 = 餌の効果 + 切片 + 誤差2 $$
$$ model3: 魚の体長 = 餌の効果 + 薬の有無 + 切片 + 誤差3 $$

よって、説明変数が多いmodelに対して、type1 ANOVAを利用することは望ましくない

In [43]:
# 説明変数を入れていないmodelとfood説明変数のmodelを比較することで餌の影響を検定
model_null = ols('length ~ 1', data=df).fit()
print(sm.stats.anova_lm(model_null, model_f, typ=1))

   df_resid            ssr  df_diff       ss_diff           F        Pr(>F)
0      99.0  166059.262534      0.0           NaN         NaN           NaN
1      98.0   74963.543872      1.0  91095.718662  119.089626  1.278970e-18


## Type2 ANOVAについて

Type2 ANOVAは、各説明変数の影響をその他の説明変数の影響を取り除いて計測するために、

$$ model1: 応答変数 = 説明A + 説明B + 説明C + 切片 + 誤差1 $$
$$ model2: 応答変数 =        説明B + 説明C + 切片 + 誤差2 $$
$$ model3: 応答変数 = 説明A        + 説明C + 切片 + 誤差3 $$
$$ model4: 応答変数 = 説明A + 説明B        + 切片 + 誤差4 $$

全て、model1とそれ以外のモデルにおいて誤差の大きさを比較することによって検定を行う

In [50]:
sm.stats.anova_lm(model_a, typ=2)

,sum_sq,df,F,PR(>F)
food,107061.631585,1.0,956.239450,4.941231e-52
medicine,64103.316067,1.0,572.549837,1.772710e-42
Residual,10860.227805,97.0,NaN,NaN
